<a href="https://colab.research.google.com/github/gazon1/Recco-challenge/blob/master/nn_with_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import json
import pandas as pd
import numpy as np


import tqdm
import scipy.sparse as sp
from sklearn.metrics import average_precision_score as score
import time
from matplotlib import pyplot as plt

from sklearn import preprocessing
from itertools import chain

from keras.models import model_from_json
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, SpatialDropout1D, Dropout, BatchNormalization, Dense,LSTM, CuDNNGRU#, CuDNNLSTM, CuDNNGRU
from keras.models import Model
from keras import optimizers

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

# Let's set random seed
import tensorflow as tf
tf.set_random_seed(42)
np.random.seed(42)



import itertools
from pprint import pprint

DATA_PATH = ''
%matplotlib inline

# For Google Colab only:
import sys
sys.path.append('/content/gdrive/My Drive/okko competition')
#from reco_utils.recommender.sar.sar_singlenode import SARSingleNode
from google.colab import drive
drive.mount('/content/gdrive')

DATA_PATH = 'gdrive/My Drive/Colab Notebooks'
CODE_PATH = 'gdrive/My Drive/okko competition/code'

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16005786974813513245
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10719191343007111468
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 17009013695528787105
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11276946637
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13043499233479227688
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
df = pd.read_csv(
    os.path.join(DATA_PATH+ 'df.csv')
)

def func(x):
  return x[1:-1].split(',')
df['gen'] = df['gen'].apply(func)

In [0]:
%%time
import tqdm
tqdm.tqdm.pandas()
sequences = df.groupby('userID').agg({'gen': sum})['gen'].progress_apply(list)
# it will take about 2 mins


100%|██████████| 496046/496046 [00:04<00:00, 103003.77it/s]


CPU times: user 1min 36s, sys: 2.4 s, total: 1min 38s
Wall time: 1min 36s


In [0]:
seqq = [sequences.values[0:,][i] for i in range(sequences.shape[0])]
seqq = np.array(seqq)

In [0]:
maxlen = 18*5 # Length of sequences in X
X = []
y = []

def slice_sequence(seq, num_slices):
  n = len(seq)
  if n > maxlen:
    X.append(seq[:maxlen-1])
    y.append(seq[maxlen])
  
        
for seq in tqdm.tqdm(seqq):
    if len(seq) <= 5*5:
        slice_sequence(seq, 2)
    elif len(seq) <= 6*5:
        slice_sequence(seq, 2)
    elif len(seq) <= 8*5:
        slice_sequence(seq, 2)
    elif len(seq) <= 12*5:
        slice_sequence(seq, 2)
    elif len(seq) <= 16:
        slice_sequence(seq, 2)
    elif len(seq) <= 20:
        slice_sequence(seq, 2)
    elif len(seq) <= 26:
        slice_sequence(seq, 2)
    else:
        slice_sequence(seq, 2)
        

# We should pad our sequences with 0 values, so they all will have the same length
X = pad_sequences(X, maxlen=maxlen,padding='post',truncating='post',value=0)
y = np.array(y)
X.shape, y.shape


100%|██████████| 496046/496046 [00:01<00:00, 268654.09it/s]


((127667, 90), (127667,))

In [0]:
X = np.ravel(X).reshape((-1, 18, 5))

In [0]:
lens = [len(x) for x in X]
max(lens), min(lens), np.mean(lens), np.median(lens)
max_features = df.categ_id.unique().size + 1
#del df

In [0]:
max_features_all = df.categ_id.unique().size + 1
maxlen=18
embed_size = 512

DEBUG = True # Выводить размерности слоев в lstm129 при вызове функции lstm129(...)

def lstm129(num_features, seq_len):
    inp = Input(shape=(seq_len,num_features))
    x = inp
    
    if DEBUG:
      print('input:', x.shape)
      
    x = BatchNormalization(epsilon=1e-6,weights=None)(x)
    x = CuDNNGRU(256,return_sequences=False)(x)
    
    if DEBUG:
      print('after CuDNNGRU:', x.shape)
      
    x = Dropout(0.02)(x)
    outp = Dense(max_features_all, activation="softmax")(x)
    
    if DEBUG:
      print('after Dense:', outp.shape)
      
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.SGD(lr=0.01, \
                                                                    momentum=0.0, decay=0.0, nesterov=True, clipnorm=1.),
                  metrics=['sparse_categorical_accuracy'])
    return model

In [0]:
le = preprocessing.LabelEncoder()


_X = np.where(np.ravel(X) == 0, np.full(np.ravel(X).shape, -1), np.ravel(X)).reshape(X.shape)
_X = np.ravel(_X)
le = preprocessing.LabelEncoder()
le = preprocessing.LabelEncoder()
all_movies = list(set(y).union(set(_X[0::5])))
le.fit(all_movies)

#теперь в _y будут числа от 0 до кол-во фильмов (max_features)
_y = le.transform(y)

_X[0::5] = le.transform(_X[0::5])
_X = _X.reshape((-1,18,5))

In [0]:
DEBUG = False
model = lstm129(5, 18)

model.fit(_X, _y, epochs=10, batch_size=2048, verbose=True, validation_split=0.01, shuffle=True)

Instructions for updating:
Use tf.cast instead.
Train on 126390 samples, validate on 1277 samples
Epoch 1/10
126390/126390 [==============================] - 8s 64us/step - loss: 9.1659 - sparse_categorical_accuracy: 5.5384e-05 - val_loss: 9.1651 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/10
 45056/126390 [=========>....................] - ETA: 4s - loss: 9.1653 - sparse_categorical_accuracy: 1.1097e-04

KeyboardInterrupt: ignored

In [0]:
sequences_test = seqq.apply(lambda x: x[-maxlen:])
sequences_test = seqq.apply(lambda x: [0 for i in range(maxlen - len(x))] + x)
test_users_in_sequences = sorted(set(sequences_test.index) & set(test_users))
X_test = np.array(sequences_test[test_users_in_sequences].tolist())

In [0]:
%%time
from itertools import chain
batch_size = 2048
n_batches = int(X_test.shape[0]/batch_size) + 1
preds = []

for batch_ind in tqdm.tqdm(range(n_batches)):
    batch = X_test[batch_ind*batch_size: (batch_ind + 1)*batch_size]
    curr_preds = model.predict(batch)
    curr_preds = np.argsort(-curr_preds)[:, :20]
    curr_preds = [[cat_to_element_uid[x] for x in row] for row in curr_preds]
    preds.append([' '.join(map(lambda x: str(x), row)) for row in curr_preds])
    
preds = list(chain(*preds))

In [0]:
top_20_videos = [2714, 747, 2639, 3783, 2245, 6127, 10061, 3916,
                 8771, 3336, 7079, 3567, 6195, 4141, 1016,
                 7185, 1004, 9467, 72, 1521]

sam_sub = dict()
for el in test_users:
    sam_sub[el] = top_20_videos

In [0]:
keys = [int(x) for x in sam_sub.keys()]
df = pd.DataFrame(keys,columns=['userID'])
df['itemID'] = df.apply(lambda x: sam_sub[x[0]],axis=1)

In [0]:
df.index = df.userID
df.itemID[test_users_in_sequences] = preds

In [0]:
ans = {}
for el, el1 in df.values:
    try:
        el1 = el1.split(' ')
        for i in range(len(el1)):
            el1[i] = int(el1[i])
        ans[str(el)]= el1
    except:
        ans[str(el)] = el1
    
with open(os.path.join(DATA_PATH, 'my_best.json'), 'w') as f:
    json.dump(ans, f)